#Step18 메모리 절약 모드
1. 역전파 시 사용하는 메모리양을 줄이는 방법 -> 불필요한 미분 결과를 보관하지 않고 즉시 삭제
2. '역전파가 필요 없는 경우용 모드'를 제공



##18.1 필요 없는 미분값 삭제

In [10]:
import numpy as np
import weakref

class Variable:
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]
    self.generation = max([x.generation for x in inputs])

    for output in outputs:
      output.set_creator(self) 
    
    self.inputs = inputs
    self.outputs= [weakref.ref(output) for output in outputs] # self.outputs가 약한 참조로 가르키도록 변경
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

class Add(Function):
  def forward(self, x0, x1):
    y = x0 + x1
    return y

  def backward(self, gy):
    '''
      backward 부분 추가
    '''
    return gy, gy

def add(x0, x1):
  return Add()(x0, x1)

class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.inputs[0].data # Function 함수에서 inputs (튜플)로 받기 때문에 input-> inputs[0]으로 바꿔줌
    gx = 2 * x * gy
    return gx

def square(x):
  return Square()(x)

# example
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)
'''
  많은 경우에서, 구하고 싶은 미분값은 말단 변수(leaf 말하는듯?)뿐 일 때가 대부분 이다.
  앞의 예에서 처럼 y와 t 같은 중간 변수의 미분 값은 필요하지 않음.
  그래서 중간 변수에 대해서는 미분 값을 제거하는 모드를 추가!
'''

1.0 1.0
2.0 1.0


'\n  많은 경우에서, 구하고 싶은 미분값은 말단 변수(leaf 말하는듯?)뿐 일 때가 대부분 이다.\n  앞의 예에서 처럼 y와 t 같은 중간 변수의 미분 값은 필요하지 않음.\n  그래서 중간 변수에 대해서는 미분 값을 제거하는 모드를 추가!\n'

In [11]:
class Variable:
  '''
  중간 변수에 대해서는 미분값을 제거하는 모드를 추가
  '''
  def __init__(self, data):
    if data is not None:
      if not isinstance(data, np.ndarray):
        raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0 # 세대 수를 기록하는 변수 추가

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1 # 세대를 기록(부모 세대 + 1)

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      if f not in seen_set:     # 같은 함수가 중복 추가 되는 것을 방지
                                # -> 두 elements가 같은 creator를 가지는 경우!
        funcs.append(f)
        seen_set.add(f)         
        funcs.sort(key=lambda x: x.generation)

    add_func(self.creator)
     
    while funcs :              
      f = funcs.pop()          
      gys = [output().grad for output in f.outputs] # output()(약한참조)의 grad 가져옴
      gxs = f.backward(*gys)                        # gxs를 구함
      if not isinstance(gxs, tuple):
        gxs = (gxs, )

      for x, gx in zip(f.inputs, gxs):            # 구한 역전파를 각 x.grad에 저장
        if x.grad is None:                        # 같은 변수일 경우에는 덮어쓰지 말고 더해주자!
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:                 # 계속해서 역전파 수행
          add_func(x.creator)                     # x.creator 추가

      if not retain_grad: # retain_grad가 false 이면 중간 변수의 미분값을 모두 None으로 설정
        for y in f.outputs:
          y().grad = None # y는 약한 참조(weakref)

def as_array(x):
  if np.isscalar(x):
    # x가 np.float64 같은 scalar 타입인지 확인(일반 float도 확인됨)
    return np.array(x)
  return x

In [8]:
# example
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)   # None None
print(x0.grad, x1.grad) # 2.0 1.0

None None
2.0 1.0


##18.2 Function 클래스 복습
-> 역전파가 필요없는 모드(inference) - 즉, 역전파를 위해 저장해야할 input이 필요 없는 모드도 있다!

In [14]:
class Config:
  '''
    enable_backprop : 역전파가 가능한지 여부를 물음
    config 데이터는 한 곳에만 존재하는 것이 좋음.
    Config 클래스는 인스턴스화 하지 않고 '클래스' 상태로 이용!
    왜냐? 인스턴스는 여러 개 생성할 수 있지만, 클래스는 항상 하나만 존재하기 때문!
  '''
  enable_backprop = True

In [15]:
class Function:
  def __call__(self, *inputs):
    xs = [x.data for x in inputs]
    ys = self.forward(*xs) # 언팩
    if not isinstance(ys, tuple): # tuple 형태가 아닐 경우 추가 지원
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop: # 클래스 상태의 enable_backprop = True 일때만 역전파 코드 실행
      self.generation = max([x.generation for x in inputs]) # 세대 설정
      for output in outputs:
        output.set_creator(self)   # 연결 설정
      self.inputs = inputs
      self.outputs= [weakref.ref(output) for output in outputs] 
    
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()


##18.4 모드 전환

In [19]:
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))

##18.3 with 문을 활용한 모드 전환
with문과 contextlib 모듈을 사용해서 구현!

In [20]:
import contextlib

@contextlib.contextmanager # 데코레이터 -> context를 판단하는 함수가 만들어짐
def config_test():
  print('start') # 전처리
  try:
    yield # yield 전에 전처리로직, 후에 후처리로직
  finally:
    print('done') # 후처리

with config_test():
  print('process...')

start
process...
done


In [21]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name) # Config의 'name'이름을 가진 속성을 가르킴
  setattr(Config, name, value) # 내가 원하는 value값으로 Config안의 name의 속성을 설정
  try:
    yield
  finally:
    setattr(Config, name, old_value) # 원래의 old value로 설정

In [23]:
with using_config('enable_backprop', False):
  '''
    Config의 enable_backprop이 False로 설정 되었으므로 -> 순전파만 실행
  '''
  x = Variable(np.array(2.0))
  y = square(x)

In [24]:
def no_grad():
  '''
    단순히 using_config('enable_backprop', False) 코드를 호출해줌
  '''
  return using_config('enable_backprop', False)

with no_grad():
  x = Variable(np.array(2.0))
  y = square(x)